In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import datetime
import os

import wire_detection_utils as wdu

In [10]:
expansion_size = 2
line_threshold = 150
pixel_binning_size = 3
low_canny_threshold = 100
high_canny_threshold = 200
bin_avg_threshold_multiplier = 0.5

# Creat e a WireDetector instance
wire_detector = wdu.WireDetector(line_threshold, expansion_size, low_canny_threshold, high_canny_threshold, pixel_binning_size, bin_avg_threshold_multiplier)

data_folder = "/media/tyler/Storage/field_tests/street_wire_experiments/wire_tracking_05-07_40fov/" 
rgb_dir = data_folder + "rgb/"
wire_detection_dir = data_folder + "wire_detection/"

if os.path.exists(wire_detection_dir) == False:
    os.makedirs(wire_detection_dir)

In [11]:

for file in os.listdir(rgb_dir):
    img = cv2.imread(rgb_dir + file)
    # Create segmentation mask
    seg_mask = wire_detector.create_seg_mask(img)
    cv2.imwrite('output_imgs/seg_mask.jpg', seg_mask)

    # Detect wires using the segmentation mask
    wire_lines, wire_midpoints, avg_angle = wire_detector.detect_wires(seg_mask)

    rand_colors = np.random.randint(0, 256, (len(wire_midpoints), 3)).tolist()
    for i, (x0,y0,x1,y1) in enumerate(wire_lines):
        cv2.line(img, (x0,y0), (x1,y1), tuple(rand_colors[i]), 1)

    for i, (x0,y0) in enumerate(wire_midpoints):
        cv2.circle(img, (int(x0), int(y0)), 5, tuple(rand_colors[i]), -1)

    large_img = cv2.resize(img, (1280, 720))
    cv2.imwrite(wire_detection_dir + file, large_img)
